# Test 6 (paper) - Experimento 4 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 6"

### Hiperparametros
* **epoch = 100**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* Mini-mask shape: 56x56
* RPN anchor scales: (32, 64, 128, 256, 512)
* Tasa de aprendizaje: 0.001
* imagenes = 292
    * entrenamiento 81% = 237
    * validacion 19% = 55
* etiquetas = 9140
* **resolucion = 1024 x 800**
* etiquetas = bounding box formato VOC XML
* numero de clases = 1 (arandano)
* **data augmentation = true**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1528
drwxr-xr-x 4 root root   4096 Aug  7 06:10 build
drwxr-xr-x 2 root root   4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root   4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  67789 Sep 16 23:55 master-test_11.ipynb
-rw-r--r-- 1 root root  88232 Sep 17 07:27 master-test_11_fase_2-Copy1.ipynb
-rw-r--r-- 1 root root  67792 Sep 17 08:36 master-test_11_fase_2.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 23:44 master-test_12.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 22:02 master-test_12_fase_2.ipynb
-rw-r--r-- 1 root root 350337 Sep 15 00:38 master-test_5-fase_2.ipynb
-rw-r--r-- 1 root root 350144 Sep 17 08:42 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root 110377 Sep 17 16:09 master-test_6.ipynb
-rw-r--r-- 1 root root 270148 Sep 17 16:09 master-test_6_fase_2.ipynb
drwxr-xr-x 4 root root   4096 Sep 13 07:49 mrcnn
drwxr-xr-x 3 root root   4096 Sep 15 02:27 old
-rw-r--r-- 1 root root 163666 Sep 18 15:38 test_5_cor

# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Entrenamiento con una sola clase y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [3]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        self.add_class("dataset", 1, "arandano")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extraer image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 11074
            # aproximadamente el 80% del conjunto de datos es para entrenamiento
            
            if is_train and int(image_id) >= 41590 :
                #vprint("image_id: ", image_id)
                continue
            
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a 11074
            if not is_train and int(image_id) < 41590:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox
        
        # <annotation>
        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>
        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances

    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
        # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # se agrega (append) el class_id 1 para arandano en nuestro caso a la variable
        class_ids = list()
        
        # se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.

        # 000000000 000000000 000001110 
        # 000011100 011100000 000001110
        # 000011100 011100000 000001110
        # 000000000 011100000 000000000
        #    1         1          1    <- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('arandano'))
        
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']


In [3]:

# clase de configuracion arandano, aqui se pueden cambiar valores de hyper parameters 
class ConfigArandanos(Config):

    # nombre de la configuracion 
    NAME = "arandano_cfg_6"    
    
    # clase arandano + clase background 
    NUM_CLASSES = 1 + 1
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    #STEPS_PER_EPOCH = 73   # por epoch se entrenaran 73 lotes de 4 imagenes, dataset = 292
    STEPS_PER_EPOCH = 146
    
    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 
    
    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [4]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [ ]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg20220907T0914/mask_rcnn_arandano_cfg_0300.h5'

conjunto_datos = 'customImages/test_6'


In [5]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()


# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)
'''
model = MaskRCNN(mode='training', model_dir='./', config=config)
'''

# cargar pesos del modelo
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sequential(iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(5/6,iaa.Fliplr(0.5), iaa.Flipud(0.5))
augmentation = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.1), iaa.Flipud(0.1))

# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='heads', augmentation=augmentation)


image_id:  31102
image_id:  40099
image_id:  30377
image_id:  40414
image_id:  21130
image_id:  50469
image_id:  41732
image_id:  25544
image_id:  50458
image_id:  23863
image_id:  30990
image_id:  30470
image_id:  20318
image_id:  50440
image_id:  21295
image_id:  22345
image_id:  40390
image_id:  20782
image_id:  30676
image_id:  30000
image_id:  50135
image_id:  50912
image_id:  22783
image_id:  20150
image_id:  40346
image_id:  31246
image_id:  25750
image_id:  50983
image_id:  50607
image_id:  25690
image_id:  40999
image_id:  21150
image_id:  23719
image_id:  11117
image_id:  20425
image_id:  40253
image_id:  51082
image_id:  20250
image_id:  41406
image_id:  20585
image_id:  11076
image_id:  50393
image_id:  30851
image_id:  50958
image_id:  50857
image_id:  25640
image_id:  51057
image_id:  50710
image_id:  31284
image_id:  40213
image_id:  40960
image_id:  50386
image_id:  41762
image_id:  41383
image_id:  22536
image_id:  30502
image_id:  50269
image_id:  30958
image_id:  222

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_5:0", shape=(Non

  3/146 [..............................] - ETA: 2:08 - batch: 1.0000 - size: 2.0000 - loss: 37.3243 - rpn_class_loss: 16.9747 - rpn_bbox_loss: 20.3495 - mrcnn_class_loss: 1.3448e-04 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00    

...

146/146 [==============================] - 271s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 12.5605 - rpn_class_loss: 2.1168 - rpn_bbox_loss: 6.8088 - mrcnn_class_loss: 0.0980 - mrcnn_bbox_loss: 3.1022 - mrcnn_mask_loss: 0.4347 - val_loss: 6.6283 - val_rpn_class_loss: 1.6787 - val_rpn_bbox_loss: 3.0988 - val_mrcnn_class_loss: 0.1368 - val_mrcnn_bbox_loss: 1.2065 - val_mrcnn_mask_loss: 0.5075
Epoch 2/100
146/146 [==============================] - 227s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 3.2932 - rpn_class_loss: 0.4383 - rpn_bbox_loss: 1.1955 - mrcnn_class_loss: 0.3060 - mrcnn_bbox_loss: 0.8922 - mrcnn_mask_loss: 0.4612 - val_loss: 3.0203 - val_rpn_class_loss: 0.6283 - val_rpn_bbox_loss: 0.8976 - val_mrcnn_class_loss: 0.2936 - val_mrcnn_bbox_loss: 0.7569 - val_mrcnn_mask_loss: 0.4439
Epoch 3/100
146/146 [==============================] - 248s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 2.2268 - rpn_class_loss: 0.2899 - rpn_bbox_loss: 0.5614 - mrcnn_class_loss: 0.3434 

Epoch 21/100
146/146 [==============================] - 227s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 1.2518 - rpn_class_loss: 0.1274 - rpn_bbox_loss: 0.2829 - mrcnn_class_loss: 0.2764 - mrcnn_bbox_loss: 0.2238 - mrcnn_mask_loss: 0.3413 - val_loss: 2.1347 - val_rpn_class_loss: 0.4622 - val_rpn_bbox_loss: 0.5574 - val_mrcnn_class_loss: 0.3199 - val_mrcnn_bbox_loss: 0.4519 - val_mrcnn_mask_loss: 0.3433
Epoch 22/100
146/146 [==============================] - 213s 1s/step - batch: 72.5000 - size: 2.0000 - loss: 1.2083 - rpn_class_loss: 0.1284 - rpn_bbox_loss: 0.2685 - mrcnn_class_loss: 0.2576 - mrcnn_bbox_loss: 0.2170 - mrcnn_mask_loss: 0.3369 - val_loss: 2.0753 - val_rpn_class_loss: 0.4567 - val_rpn_bbox_loss: 0.5228 - val_mrcnn_class_loss: 0.3397 - val_mrcnn_bbox_loss: 0.3838 - val_mrcnn_mask_loss: 0.3723
Epoch 23/100
146/146 [==============================] - 210s 1s/step - batch: 72.5000 - size: 2.0000 - loss: 1.2064 - rpn_class_loss: 0.1268 - rpn_bbox_loss: 0.2743 - mrcnn_class

Epoch 41/100
146/146 [==============================] - 239s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.9466 - rpn_class_loss: 0.0893 - rpn_bbox_loss: 0.1990 - mrcnn_class_loss: 0.1915 - mrcnn_bbox_loss: 0.1577 - mrcnn_mask_loss: 0.3090 - val_loss: 2.2438 - val_rpn_class_loss: 0.4906 - val_rpn_bbox_loss: 0.5553 - val_mrcnn_class_loss: 0.4798 - val_mrcnn_bbox_loss: 0.3575 - val_mrcnn_mask_loss: 0.3606
Epoch 42/100
146/146 [==============================] - 230s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.9311 - rpn_class_loss: 0.0832 - rpn_bbox_loss: 0.1948 - mrcnn_class_loss: 0.1918 - mrcnn_bbox_loss: 0.1566 - mrcnn_mask_loss: 0.3047 - val_loss: 2.3996 - val_rpn_class_loss: 0.6328 - val_rpn_bbox_loss: 0.5360 - val_mrcnn_class_loss: 0.5241 - val_mrcnn_bbox_loss: 0.3571 - val_mrcnn_mask_loss: 0.3495
Epoch 43/100
146/146 [==============================] - 237s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.9536 - rpn_class_loss: 0.0930 - rpn_bbox_loss: 0.2037 - mrcnn_class

Epoch 61/100
146/146 [==============================] - 229s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7718 - rpn_class_loss: 0.0725 - rpn_bbox_loss: 0.1539 - mrcnn_class_loss: 0.1372 - mrcnn_bbox_loss: 0.1214 - mrcnn_mask_loss: 0.2869 - val_loss: 2.4779 - val_rpn_class_loss: 0.6169 - val_rpn_bbox_loss: 0.5394 - val_mrcnn_class_loss: 0.6046 - val_mrcnn_bbox_loss: 0.3675 - val_mrcnn_mask_loss: 0.3494
Epoch 62/100
146/146 [==============================] - 239s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7888 - rpn_class_loss: 0.0753 - rpn_bbox_loss: 0.1688 - mrcnn_class_loss: 0.1343 - mrcnn_bbox_loss: 0.1220 - mrcnn_mask_loss: 0.2885 - val_loss: 2.2898 - val_rpn_class_loss: 0.5363 - val_rpn_bbox_loss: 0.5709 - val_mrcnn_class_loss: 0.4664 - val_mrcnn_bbox_loss: 0.3610 - val_mrcnn_mask_loss: 0.3552
Epoch 63/100
146/146 [==============================] - 218s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7276 - rpn_class_loss: 0.0635 - rpn_bbox_loss: 0.1384 - mrcnn_class

Epoch 81/100
146/146 [==============================] - 253s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.6271 - rpn_class_loss: 0.0489 - rpn_bbox_loss: 0.1177 - mrcnn_class_loss: 0.0985 - mrcnn_bbox_loss: 0.0955 - mrcnn_mask_loss: 0.2665 - val_loss: 2.6169 - val_rpn_class_loss: 0.8295 - val_rpn_bbox_loss: 0.5569 - val_mrcnn_class_loss: 0.5483 - val_mrcnn_bbox_loss: 0.3502 - val_mrcnn_mask_loss: 0.3321
Epoch 82/100
146/146 [==============================] - 223s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.6334 - rpn_class_loss: 0.0513 - rpn_bbox_loss: 0.1180 - mrcnn_class_loss: 0.0990 - mrcnn_bbox_loss: 0.0943 - mrcnn_mask_loss: 0.2708 - val_loss: 2.7102 - val_rpn_class_loss: 0.7791 - val_rpn_bbox_loss: 0.6010 - val_mrcnn_class_loss: 0.6344 - val_mrcnn_bbox_loss: 0.3505 - val_mrcnn_mask_loss: 0.3453
Epoch 83/100
146/146 [==============================] - 241s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.6385 - rpn_class_loss: 0.0544 - rpn_bbox_loss: 0.1189 - mrcnn_class